In [46]:
import tifffile
from skimage.morphology import white_tophat, disk
from skimage.filters import gaussian
from skimage.measure import label, regionprops_table
from skimage.util import map_array
import numpy as np
import napari
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
viewer = napari.Viewer()

Use similar analysis (rolling ball background subtraction, gaussian blur, threshold, label, regionprops_table) to make a label image of the nucleii in "Easy.tif". 

In [3]:
easy = tifffile.imread("/Users/cjw/Desktop/pythonClass/zyu/files/Easy.tif")
easy.shape

(512, 512)

In [4]:
viewer.add_image(easy, colormap="bop blue")

<Image layer 'easy' at 0x113391d50>

In [21]:
g0 = white_tophat(easy, disk(20))
viewer.add_image(g0)

<Image layer 'g0' at 0x2ef6e3400>

In [23]:
g1 = gaussian(g0, 1, preserve_range=True)
viewer.add_image(g1)

<Image layer 'g1' at 0x2f289e1a0>

In [32]:
fig = px.line(y=easy[310, :])
fig.add_scatter(y=g0[310, :], name="background subtraction")
fig.add_scatter(y=g1[310, :], name="smoothed")

In [26]:
tx = g1 > 25

In [29]:
viewer.add_image(tx)

<Image layer 'tx' at 0x2f29f32e0>

In [30]:
b = label(tx)
viewer.add_labels(b)

<Labels layer 'b' at 0x2ef5e63e0>

In [43]:
props_dict = regionprops_table(b, easy, properties=['label', 'area', 'centroid', 'axis_major_length', 
                                                    'axis_minor_length'])

propsdf = pd.DataFrame(props_dict)

In [45]:
propsdf.head()

,label,area,centroid-0,centroid-1,axis_major_length,axis_minor_length
0,1,63.0,2.730159,30.428571,10.709156,7.647223
1,2,72.0,5.166667,47.250000,12.346121,7.417394
2,3,9.0,0.333333,423.333333,5.971099,1.859326
3,4,232.0,7.560345,444.909483,28.805535,11.811016
4,5,17.0,1.882353,509.882353,6.446564,3.547836


In [51]:
areas =  map_array(b, propsdf.label.values, propsdf.area.values)


In [53]:
viewer.add_image(areas)

<Image layer 'areas' at 0x311b55900>

In [57]:
px.histogram(propsdf, x='area', nbins=50)

In [56]:
filtered = propsdf[propsdf['area'] < 200]

In [61]:
small_img = np.where(areas < 175, b, 0)

In [62]:
viewer.add_image(small_img)

<Image layer 'small_img [1]' at 0x3126e51e0>